In [1]:
%%bash
nvidia-smi

Thu Dec 18 15:08:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:1A:00.0 Off |                  Off |
| 30%   42C    P2             155W / 300W |  22297MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import time
import sys

def occupy_gpu_memory(gpu_ids, target_gb_per_card):
    """
    在指定的 GPU 上占用固定大小的显存
    """
    tensors = []
    
    for gpu_id in gpu_ids:
        try:
            # 切换到目标显卡
            device = torch.device(f"cuda:{gpu_id}")
            
            # 计算需要申请的 Tensor 大小 (float32 占用 4 bytes)
            # 1GB = 1024 * 1024 * 1024 bytes
            total_elements = (target_gb_per_card * 1024 * 1024 * 1024) // 4
            
            print(f"正在为 GPU {gpu_id} 申请 {target_gb_per_card} GB 显存...")
            
            # 申请显存并搬运到 GPU
            dummy_tensor = torch.empty(int(total_elements), dtype=torch.float32, device=device).fill_(0.0)
            tensors.append(dummy_tensor)
            
        except RuntimeError as e:
            print(f"GPU {gpu_id} 申请失败: {e}")

    print("\n[成功] 已占用指定显存。按下 Ctrl+C 停止占用并释放。")
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n正在释放显存并退出...")

if __name__ == "__main__":
    # 指定要占用的显卡 ID
    target_gpus = [1,2,3,4,5,6,7,8]
    
    # 指定每张卡占用的 GB 数 (根据你 3090/4090 24GB 的情况，建议留出 4-6GB 给系统和基础开销)
    # 比如占用 18GB
    gb_to_occupy = 20
    
    occupy_gpu_memory(target_gpus, gb_to_occupy)

正在为 GPU 1 申请 20 GB 显存...
GPU 1 申请失败: CUDA out of memory. Tried to allocate 20.00 GiB. GPU 1 has a total capacity of 23.65 GiB of which 15.10 GiB is free. Process 134954 has 966.00 MiB memory in use. Process 134959 has 966.00 MiB memory in use. Process 134946 has 966.00 MiB memory in use. Process 134958 has 966.00 MiB memory in use. Process 134945 has 966.00 MiB memory in use. Process 134949 has 966.00 MiB memory in use. Process 134951 has 966.00 MiB memory in use. Process 134953 has 966.00 MiB memory in use. Process 256495 has 590.00 MiB memory in use. Including non-PyTorch memory, this process has 384.00 MiB memory in use. Of the allocated memory 0 bytes is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
正在为 GPU 2 申请 20 